This is a exercise following [this blog from Medium](https://towardsdatascience.com/value-investing-dashboard-with-python-beautiful-soup-and-dash-python-43002f6a97ca).

The goal of this application is to help you retrieve and display the right financial insights quickly about a certain company ticker.

Tool list:
- Beautiful Soup as the Web Scraping Tool
- Pandas data-reader API to get the financial stocks price
- Flask/Dash for Web Application Framework

Install follow packages to start using Dash :
- The core dash backend.
- Dash front-end
- Dash HTML components
- Dash core components
- Plotly

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [19]:
### Extract lists of the stocks with their tickers
def save_sp500_stocks_info():  
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = BeautifulSoup(resp.text,'lxml')
    table = soup.find('table',{'class':'wikitable sortable'})
    stocks_info=[]
    tickers, securities, gics_industries, gics_sub_industries = [],[],[],[]
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        security = row.findAll('td')[1].text
        gics_industry = row.findAll('td')[3].text
        gics_sub_industry = row.findAll('td')[4].text
        
        tickers.append(ticker.lower().replace(r"\n"," "))
        securities.append(security)
        gics_industries.append(gics_industry.lower())
        gics_sub_industries.append(gics_sub_industry.lower())

    stocks_info.append(tickers)
    stocks_info.append(securities)
    stocks_info.append(gics_industries)
    stocks_info.append(gics_sub_industries)

    stocks_info_df = pd.DataFrame(stocks_info).T
    stocks_info_df.columns=['tickers','security','gics_industry','gics_sub_industry']
    stocks_info_df['seclabels'] = 'SP500'
    stocks_info_df['labels'] = stocks_info_df[['tickers','security', 'gics_industry','gics_sub_industry','seclabels']].apply(lambda x: ' '.join(x), axis=1)

    dictlist = []
    for index, row in stocks_info_df.iterrows():
        dictlist.append({'value':row['tickers'], 'label':row['labels']})
    return dictlist

In [27]:
# Tickers from russell
# This function will return empty array until the csv file of Russell components is available.
def save_russell_info():
    print("Getting russell stocks info")
    russell_csv_url = None
    dictlist = []
    if (russell_csv_url != None):
        dfrussel=pd.read_csv(russell_csv_url,index_col='Symbol')
        dfrussel['tickers'] = dfrussel.index.str.upper()
        dfrussel['tickers'] = dfrussel['tickers'].replace(r"\n", " ")
        dfrussel['security'] = dfrussel.Description.str.title()
        dfrussel['gics_industry'] = dfrussel.Sector.str.lower()
        dfrussel['gics_sub_industry'] = dfrussel.Industry.str.lower()
        dfrussel['seclabels'] = 'RUSSELL'

        dfrussel['labels'] = dfrussel[['tickers','security','gics_industry','gics_sub_industry','seclabels']].apply(lambda x: ' '.join(x), axis=1)

        for index, row in dfrussel.iterrows():
            dictlist.append({'value':row['tickers'], 'label':row['labels']})
    else:
        print("Warning ! Russell stock list csv file not found !")

    return dictlist

In [29]:
# self append
def save_self_stocks_info():
    print("Adding own list of stocks info")

    dictlist = []

    dictlist.append({'value':'ajbu', 'label':'AJBU Keppel DC Reit Data REITS SA'})
    dictlist.append({'value':'gme', 'label':'GME Game Stop Corp SA'})
    dictlist.append({'value':'aeg', 'label':'AEG Aegon Insurance SA'})
    dictlist.append({'value':'ntic', 'label':'NTIC Northern Technologies International SA'})
    dictlist.append({'value':'sq', 'label':'SQ Square SA'})
    dictlist.append({'value':'kbsty', 'label':'Kobe steel'})
    dictlist.append({'value':'NESN', 'label':'Nestle'})
    dictlist.append({'value':'BN', 'label':'Danone'})
    dictlist.append({'value': 'DATA', 'label': 'Tableau Software Data Visualization'})
    dictlist.append({'value': 'S58','label':'SATS'})

    return dictlist